In [1]:
%pip install scikit-learn pandas numpy matplotlib tabulate


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.8 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import re
pd.set_option("display.max_columns", None)
%cd /mnt

2023-07-22 10:44:27.255463: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-22 10:44:27.281353: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


/mnt


In [3]:
def unzip(path_to_zip_file, directory_to_extract_to):
    """
    Args:
        path_to_zip_file (str): 압축파일 경로
        directory_to_extract_to (str): 압축 풀고 저장할 경로
    """
    import zipfile
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)


In [4]:
data = pd.read_csv("Full-Dataset/processed/url_encoded_dataset.csv").sample(frac=1)
data = data.drop(["Unnamed: 0", "http.request.uri.query", "http.file_data", "http.request.full_uri", 
                  "tcp.options", "http.request.uri.path", "http.request.uri.query.dict", "http.file_data.dict", "http.request.uri.scheme"], axis="columns")
data["http.data"] = data["http.data"].fillna("")
original_data = data.copy()

# RNN학습시, URL정보를 바탕으로 분류를 진행할 수 있으므로 URL 정보를 삭제해야함.
data["tcp.payload"] = original_data["tcp.payload"].astype("str").apply(lambda x: " ".join([_ for _ in x.split(" ") if not (_.startswith("/DVWA") and _.endswith(".php"))]))

/tmp/ipykernel_11719/524550114.py:1: DtypeWarning: Columns (10,14,19,27) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("Full-Dataset/processed/url_encoded_dataset.csv").sample(frac=1)


## RNN을 위한 전처리

In [5]:
# Tokenize values
tokenizer = Tokenizer(oov_token="<NUMBER>", filters='!"#$%&()*+,-./:?@[\\]^_`{|}~\t\n=',)
tokenizer.fit_on_texts(original_data["http.data"])
padded = pad_sequences(tokenizer.texts_to_sequences(original_data["http.data"]), padding="post", value=0)
data["http.data"] = padded.tolist()

In [6]:
word_index_dict = {val: key for key, val in tokenizer.word_index.items()}
pd.Series(tokenizer.texts_to_sequences(original_data["http.data"])).apply(lambda x: ", ".join([(word_index_dict)[_] for _ in x])).unique()

array(['',
       '0, 0, id, 8, 20and, 209590, 3d, 28select, 20upper, 28xmltype, 28chr, 2860, 29, 7c, 7cchr, 2858, 29, 7c, 7cchr, 28113, 29, 7c, 7cchr, 28106, 29, 7c, 7cchr, 2898, 29, 7c, 7cchr, 2898, 29, 7c, 7cchr, 28113, 29, 7c, 7c, 28select, 20, 28case, 20when, 20, 289590, 3d9590, 29, 20then, 201, 20else, 200, 20end, 29, 20from, 20dual, 29, 7c, 7cchr, 28113, 29, 7c, 7cchr, 28112, 29, 7c, 7cchr, 28113, 29, 7c, 7cchr, 28120, 29, 7c, 7cchr, 28113, 29, 7c, 7cchr, 2862, 29, 29, 29, 20from, 20dual, 29, submit, submitid, 58',
       '<, doctype, html, public, ietf, dtd, html, 2, 0, en, >, n<html><head>, n<title>404, not, found<, title>, n<, head><body>, n<h1>not, found<, h1>, n<p>the, requested, url, was, not, found, on, this, server, <, p>, n<hr>, n<address>apache, 2, 4, 38, raspbian, server, at, 192, 168, 0, 128, port, 80<, address>, n<, body><, html>, n',
       ...,
       '0, 0, id, 3, submit, submitid, 3d54, 27, 20and, 20extractvalue, 288809, 2cconcat, 280x5c, 2c0x716b6a6271, 2c, 28s

In [7]:
# Tokenize tcp.payload
tokenizer_payload = Tokenizer(oov_token="<PAD>")
tokenizer_payload.fit_on_texts(original_data["tcp.payload"].astype("str").apply(lambda x: " ".join([x.split(" ")[0],*x.split(" ")[2:]])))
padded = pad_sequences(tokenizer_payload.texts_to_sequences(original_data["tcp.payload"].astype("str")), padding="post", value=0)
data["tcp.payload"] = padded.tolist()

In [8]:
data.head()

,tcp.len,tcp.checksum,tcp.ack,http.response,tcp.connection.fin,ip.dst_host,http.content_length,Attack_type,arp.opcode,tcp.srcport,tcp.payload,arp.dst.proto_ipv4,tcp.connection.synack,tcp.seq,http.referer,frame.time,tcp.dstport,tcp.connection.syn,tcp.connection.rst,ip.src_host,tcp.ack_raw,tcp.flags,arp.src.proto_ipv4,tcp.flags.ack,arp.hw.size,0,GET,POST,TRACE,HTTP/1.0,HTTP/1.1,http.request.uri.netloc,http.data
19648,86.0,0x00005ca5,1.0,0.0,0.0,192.168.0.128,0.0,Password,0.0,53072.0,"[15, 1, 27, 1, 5, 4, 2, 9, 6, 7, 2, 8, 10, 11,...",0,0.0,1.0,0.0,2021 17:44:55.977845000,80.0,0.0,0.0,192.168.0.170,1.707207e+09,0x00000018,0,1.0,0.0,False,True,False,False,True,False,192.168.0.128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
40399,221.0,0x0000308d,1.0,0.0,0.0,192.168.0.128,0.0,Password,0.0,55420.0,"[15, 1, 27, 1, 5, 4, 2, 9, 6, 7, 2, 8, 10, 11,...",0,0.0,1.0,0.0,2021 19:04:23.669533000,80.0,0.0,0.0,192.168.0.170,1.380546e+09,0x00000018,0,1.0,0.0,False,True,False,False,True,False,192.168.0.128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
51206,86.0,0x00007429,1.0,0.0,0.0,192.168.0.128,0.0,Password,0.0,57776.0,"[15, 1, 27, 1, 5, 4, 2, 9, 6, 7, 2, 8, 10, 11,...",0,0.0,1.0,0.0,2021 19:41:15.021307000,80.0,0.0,0.0,192.168.0.170,3.861345e+09,0x00000018,0,1.0,0.0,False,True,False,False,True,False,192.168.0.128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
36759,86.0,0x00003abe,1.0,0.0,0.0,192.168.0.128,0.0,Password,0.0,45096.0,"[15, 1, 27, 1, 5, 4, 2, 9, 6, 7, 2, 8, 10, 11,...",0,0.0,1.0,0.0,2021 18:52:11.865607000,80.0,0.0,0.0,192.168.0.170,3.245930e+08,0x00000018,0,1.0,0.0,False,True,False,False,True,False,192.168.0.128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
10543,221.0,0x00007efe,1.0,0.0,0.0,192.168.0.128,0.0,Password,0.0,55432.0,"[15, 1, 27, 1, 5, 4, 2, 9, 6, 7, 2, 8, 10, 11,...",0,0.0,1.0,0.0,2021 17:14:34.076578000,80.0,0.0,0.0,192.168.0.170,2.280952e+08,0x00000018,0,1.0,0.0,False,True,False,False,True,False,192.168.0.128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [9]:
original_data.head()

,tcp.len,tcp.checksum,tcp.ack,http.response,tcp.connection.fin,ip.dst_host,http.content_length,Attack_type,arp.opcode,tcp.srcport,tcp.payload,arp.dst.proto_ipv4,tcp.connection.synack,tcp.seq,http.referer,frame.time,tcp.dstport,tcp.connection.syn,tcp.connection.rst,ip.src_host,tcp.ack_raw,tcp.flags,arp.src.proto_ipv4,tcp.flags.ack,arp.hw.size,0,GET,POST,TRACE,HTTP/1.0,HTTP/1.1,http.request.uri.netloc,http.data
19648,86.0,0x00005ca5,1.0,0.0,0.0,192.168.0.128,0.0,Password,0.0,53072.0,GET /DVWA/login.php HTTP/1.0\nHost: 192.168.0....,0,0.0,1.0,0.0,2021 17:44:55.977845000,80.0,0.0,0.0,192.168.0.170,1.707207e+09,0x00000018,0,1.0,0.0,False,True,False,False,True,False,192.168.0.128,
40399,221.0,0x0000308d,1.0,0.0,0.0,192.168.0.128,0.0,Password,0.0,55420.0,GET /DVWA/login.php HTTP/1.0\nHost: 192.168.0....,0,0.0,1.0,0.0,2021 19:04:23.669533000,80.0,0.0,0.0,192.168.0.170,1.380546e+09,0x00000018,0,1.0,0.0,False,True,False,False,True,False,192.168.0.128,
51206,86.0,0x00007429,1.0,0.0,0.0,192.168.0.128,0.0,Password,0.0,57776.0,GET /DVWA/login.php HTTP/1.0\nHost: 192.168.0....,0,0.0,1.0,0.0,2021 19:41:15.021307000,80.0,0.0,0.0,192.168.0.170,3.861345e+09,0x00000018,0,1.0,0.0,False,True,False,False,True,False,192.168.0.128,
36759,86.0,0x00003abe,1.0,0.0,0.0,192.168.0.128,0.0,Password,0.0,45096.0,GET /DVWA/login.php HTTP/1.0\nHost: 192.168.0....,0,0.0,1.0,0.0,2021 18:52:11.865607000,80.0,0.0,0.0,192.168.0.170,3.245930e+08,0x00000018,0,1.0,0.0,False,True,False,False,True,False,192.168.0.128,
10543,221.0,0x00007efe,1.0,0.0,0.0,192.168.0.128,0.0,Password,0.0,55432.0,GET /DVWA/login.php HTTP/1.0\nHost: 192.168.0....,0,0.0,1.0,0.0,2021 17:14:34.076578000,80.0,0.0,0.0,192.168.0.170,2.280952e+08,0x00000018,0,1.0,0.0,False,True,False,False,True,False,192.168.0.128,


In [10]:
def zipFile(fileName):
    import shutil
    shutil.make_archive(fileName,"zip", fileName)

## DNN을 위한 전처리

In [11]:
# process time data into hour
def time_to_sec(time):
    hour, minute, second = list(map(float, time[5:].strip().split(":")))
    sec = hour*3600 + minute*60 + second
    return sec / 3600
data["frame.time"] = original_data["frame.time"].apply(time_to_sec)

In [12]:
# tcp.checksum float로 변경
data["tcp.checksum"] = original_data["tcp.checksum"].astype("str").apply(lambda x: int(x[2:] if x[2:] else "0", 16))

In [13]:
# tcp.flag float로 변경
data["tcp.flags"] = original_data["tcp.flags"].astype("str").apply(lambda x: int(x[2:] if x[2:] else "0", 16))

In [14]:
# tcp.ack_raw 정규화
data["tcp.ack_raw"] = (original_data["tcp.ack_raw"] - original_data["tcp.ack_raw"].mean()) / original_data["tcp.ack_raw"].std()

In [15]:
# tcp.ack 정규화
data["tcp.ack"] = (original_data["tcp.ack"] - original_data["tcp.ack"].mean()) / original_data["tcp.ack"].std()

In [16]:
for feature in ["GET", "POST", "TRACE", "HTTP/1.0", "HTTP/1.1"]:
    data[feature] = original_data[feature].apply(int)

In [17]:
# http.referer는 [0.0, '0.0', '0', '127.0.0.1'] 의 값만을 갖고 있으므로, 의미가 없음.
data = data.drop(["http.referer"], axis="columns") 

In [18]:
data = data.drop(["0"], axis="columns")

In [19]:
# DNN을 할 columns
DNN_COLS = ['frame.time', 'tcp.checksum', 'http.response',
       'http.content_length',
       'tcp.connection.rst', 'arp.hw.size', 'tcp.len',
       'tcp.seq', 'tcp.connection.syn',
       'tcp.flags.ack', 'tcp.ack_raw', 'tcp.connection.fin', 'tcp.dstport',
       'tcp.flags', 'tcp.ack', 'tcp.connection.synack', 
       'tcp.srcport', 'arp.opcode', 'GET', 'POST', 'TRACE', 'HTTP/1.0',
       'HTTP/1.1']
data[DNN_COLS].head(10)

,frame.time,tcp.checksum,http.response,http.content_length,tcp.connection.rst,arp.hw.size,tcp.len,tcp.seq,tcp.connection.syn,tcp.flags.ack,tcp.ack_raw,tcp.connection.fin,tcp.dstport,tcp.flags,tcp.ack,tcp.connection.synack,tcp.srcport,arp.opcode,GET,POST,TRACE,HTTP/1.0,HTTP/1.1
19648,17.748883,23717,0.0,0.0,0.0,0.0,86.0,1.0,0.0,1.0,-0.300375,0.0,80.0,24,-0.263640,0.0,53072.0,0.0,1,0,0,1,0
40399,19.073242,12429,0.0,0.0,0.0,0.0,221.0,1.0,0.0,1.0,-0.557081,0.0,80.0,24,-0.263640,0.0,55420.0,0.0,1,0,0,1,0
51206,19.687506,29737,0.0,0.0,0.0,0.0,86.0,1.0,0.0,1.0,1.392455,0.0,80.0,24,-0.263640,0.0,57776.0,0.0,1,0,0,1,0
36759,18.869963,15038,0.0,0.0,0.0,0.0,86.0,1.0,0.0,1.0,-1.386902,0.0,80.0,24,-0.263640,0.0,45096.0,0.0,1,0,0,1,0
10543,17.242799,32510,0.0,0.0,0.0,0.0,221.0,1.0,0.0,1.0,-1.462734,0.0,80.0,24,-0.263640,0.0,55432.0,0.0,1,0,0,1,0
83407,18.612451,13237,0.0,0.0,0.0,0.0,102.0,103.0,0.0,1.0,-1.623860,0.0,80.0,24,3.674564,0.0,59858.0,0.0,1,0,0,0,1
74275,21.694433,38679,0.0,0.0,0.0,0.0,677.0,1.0,0.0,1.0,1.071944,0.0,80.0,24,-0.263640,0.0,43528.0,0.0,1,0,0,0,1
20072,17.774122,32180,0.0,0.0,0.0,0.0,221.0,1.0,0.0,1.0,-1.175533,0.0,80.0,24,-0.263640,0.0,54270.0,0.0,1,0,0,1,0
51086,19.680961,57095,0.0,0.0,0.0,0.0,86.0,1.0,0.0,1.0,-1.199176,0.0,80.0,24,-0.263640,0.0,57440.0,0.0,1,0,0,1,0
61896,20.276736,22316,0.0,0.0,0.0,0.0,221.0,1.0,0.0,1.0,1.197410,0.0,80.0,24,-0.263640,0.0,59934.0,0.0,1,0,0,1,0


## 데이터 분할

In [20]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.3, random_state=42)

In [21]:
labels = {key: i for i, key in enumerate(sorted(test["Attack_type"].unique()))}
def to_onehot(x):
    l = [0 for _ in range(labels.__len__())]
    l[labels[x]] = 1
    return l
test_X, test_Y = test.drop(["Attack_type"], axis="columns"), np.stack(test["Attack_type"].apply(to_onehot))
train_X, train_Y =  train.drop(["Attack_type"], axis="columns"), np.stack(train["Attack_type"].apply(to_onehot))

## RNN 실행

In [22]:
import keras.backend as K
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


In [23]:
payload_test_X ,RNN_test_X =  np.stack(test["tcp.payload"]),np.stack(test["http.data"])
payload_train_X ,RNN_train_X =  np.stack(train["tcp.payload"]),np.stack(train["http.data"])

In [24]:
file_data_vocab_size = tokenizer.word_index.keys().__len__() + 1
file_data_input_dim = RNN_train_X.shape[1]

In [25]:
import os

RNN_MODEL_PATH:str = "http_data_RNN.model"
FORCE_TRAIN = True
layers = [
            keras.layers.Embedding(file_data_vocab_size, 128, input_length=file_data_input_dim),
            keras.layers.LSTM(128, dropout=0.7, activation="tanh"),
            keras.layers.Dense(100, activation=tf.nn.relu),
            keras.layers.Dense(4, activation='softmax'),
        ]

compileArgs = {
    "loss": tf.keras.losses.CategoricalCrossentropy(), 
    "optimizer": tf.keras.optimizers.Adam(learning_rate=.005), 
    "metrics": ["accuracy", f1_score, precision, recall],
}

if not FORCE_TRAIN and "http_data_RNN.model.zip" in os.listdir(os.getcwd()):
    unzip(f"{RNN_MODEL_PATH}.zip", f"{RNN_MODEL_PATH}")
    RNN_model = keras.models.load_model(f"{RNN_MODEL_PATH}")
else:
    RNN_model = keras.Sequential(
        layers = layers.copy()
    )
    RNN_model.compile(
        **compileArgs
    )
    RNN_model.fit(RNN_train_X, train_Y, epochs=15, batch_size=128)
    RNN_model.save("http_data_RNN.model")
    zipFile("http_data_RNN.model")

2023-07-22 10:44:35.155091: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-22 10:44:35.159518: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-22 10:44:35.159791: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-22 10:44:35.162818: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-22 10:44:35.163172: I tensorflow/compile

Epoch 1/15


2023-07-22 10:44:37.796208: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-07-22 10:44:37.883747: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-07-22 10:44:37.886493: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x45de0050 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-22 10:44:37.886516: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2023-07-22 10:44:37.889862: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-22 10:44:37.967992: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime o

510/510 [==============================] - 21s 38ms/step - loss: 0.5660 - accuracy: 0.8070 - f1_score: 0.8040 - precision: 0.8041 - recall: 0.8040
Epoch 2/15
510/510 [==============================] - 7s 15ms/step - loss: 0.5444 - accuracy: 0.8161 - f1_score: 0.8155 - precision: 0.8175 - recall: 0.8148
Epoch 3/15
510/510 [==============================] - 6s 13ms/step - loss: 0.3682 - accuracy: 0.8414 - f1_score: 0.8414 - precision: 0.8414 - recall: 0.8414
Epoch 4/15
510/510 [==============================] - 7s 13ms/step - loss: 0.3682 - accuracy: 0.8414 - f1_score: 0.8414 - precision: 0.8414 - recall: 0.8414
Epoch 5/15
510/510 [==============================] - 5s 11ms/step - loss: 0.3675 - accuracy: 0.8414 - f1_score: 0.8414 - precision: 0.8414 - recall: 0.8414
Epoch 6/15
510/510 [==============================] - 6s 12ms/step - loss: 0.3668 - accuracy: 0.8414 - f1_score: 0.8414 - precision: 0.8414 - recall: 0.8414
Epoch 7/15
510/510 [==============================] - 6s 11ms/step -

INFO:tensorflow:Assets written to: http_data_RNN.model/assets


In [26]:
payload_vocab_size = tokenizer_payload.word_index.keys().__len__() + 1
payload_input_dim = payload_train_X.shape[1]

In [27]:
PAYLOAD_RNN_MODEL_PATH:str = "tcp_payload_RNN.model"
FORCE_TRAIN = True
if not FORCE_TRAIN and "tcp_payload_RNN.model.zip" in os.listdir(os.getcwd()):
    unzip(f"{PAYLOAD_RNN_MODEL_PATH}.zip", f"{PAYLOAD_RNN_MODEL_PATH}")
    payload_RNN_model = keras.models.load_model(f"{PAYLOAD_RNN_MODEL_PATH}")
else:
    payload_RNN_model = keras.Sequential(
        layers = [
            keras.layers.Embedding(payload_vocab_size, 128, input_length=payload_input_dim),
            keras.layers.LSTM(128, dropout=0.7),
            keras.layers.Dense(100, activation="relu"),
            keras.layers.Dense(4, activation='softmax'),
        ]
    )
    payload_RNN_model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(), 
        optimizer="adam", 
        metrics=["accuracy", f1_score, precision, recall],
    )
    payload_RNN_model.fit(payload_train_X, train_Y, epochs=10, batch_size=128)
    payload_RNN_model.save("tcp_payload_RNN.model")
    zipFile("tcp_payload_RNN.model")

Epoch 1/10
510/510 [==============================] - 16s 30ms/step - loss: 0.5036 - accuracy: 0.8372 - f1_score: 0.8299 - precision: 0.8329 - recall: 0.8273
Epoch 2/10
510/510 [==============================] - 8s 16ms/step - loss: 0.0691 - accuracy: 0.9792 - f1_score: 0.9792 - precision: 0.9792 - recall: 0.9792
Epoch 3/10
510/510 [==============================] - 7s 14ms/step - loss: 0.0680 - accuracy: 0.9793 - f1_score: 0.9793 - precision: 0.9793 - recall: 0.9793
Epoch 4/10
510/510 [==============================] - 7s 14ms/step - loss: 0.0688 - accuracy: 0.9792 - f1_score: 0.9792 - precision: 0.9792 - recall: 0.9792
Epoch 5/10
510/510 [==============================] - 7s 14ms/step - loss: 0.0681 - accuracy: 0.9793 - f1_score: 0.9793 - precision: 0.9793 - recall: 0.9793
Epoch 6/10
510/510 [==============================] - 7s 14ms/step - loss: 0.0706 - accuracy: 0.9789 - f1_score: 0.9789 - precision: 0.9789 - recall: 0.9789
Epoch 7/10
510/510 [==============================] - 7s 

INFO:tensorflow:Assets written to: tcp_payload_RNN.model/assets


In [28]:
file_data_prediction = RNN_model.predict(RNN_test_X)

875/875 [==============================] - 3s 3ms/step


In [29]:
payload_prediction = payload_RNN_model.predict(payload_test_X)

875/875 [==============================] - 3s 4ms/step


In [30]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, f1_score

In [31]:
pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(file_data_prediction, axis=1)),index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys()))

,PRED_Password,PRED_SQL_injection,PRED_Uploading,PRED_XSS
TRUE_Password,21366,0,0,0
TRUE_SQL_injection,0,1291,0,0
TRUE_Uploading,0,0,568,0
TRUE_XSS,4419,0,0,327


In [32]:
pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(payload_prediction, axis=1)),index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys()))

,PRED_Password,PRED_SQL_injection,PRED_Uploading,PRED_XSS
TRUE_Password,21366,0,0,0
TRUE_SQL_injection,0,1291,0,0
TRUE_Uploading,0,0,0,568
TRUE_XSS,2,0,0,4744


## DNN 실행

In [33]:
import keras.backend as K
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


In [34]:
DNN_layers = [
        keras.layers.Dense(4196, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(len(labels.keys()), activation="softmax"),
    ]
DNN_compile = {
    "loss": tf.keras.losses.CategoricalCrossentropy(), 
    "optimizer": "adam", 
    "metrics": ["accuracy", f1_score, precision, recall],
}
predictions = []
DNN_models = [keras.Sequential(layers=DNN_layers.copy()) for _ in [32, 64, 128, 256, 512]]
for DNN_model,b_size in zip(DNN_models, [32, 64, 128, 256, 512]):
    DNN_model.compile(**DNN_compile)
    DNN_model.fit(train_X[DNN_COLS], train_Y, epochs=20, batch_size=b_size)
    DNN_prediction = DNN_model.predict(test_X[DNN_COLS])
    predictions.append(DNN_prediction)

Epoch 1/20


2040/2040 [==============================] - 15s 6ms/step - loss: 6.5186 - accuracy: 0.8315 - f1_score: 0.8288 - precision: 0.8342 - recall: 0.8243
Epoch 2/20
2040/2040 [==============================] - 13s 6ms/step - loss: 0.2731 - accuracy: 0.9236 - f1_score: 0.9232 - precision: 0.9248 - recall: 0.9216
Epoch 3/20
2040/2040 [==============================] - 13s 6ms/step - loss: 0.5047 - accuracy: 0.9095 - f1_score: 0.9093 - precision: 0.9102 - recall: 0.9085
Epoch 4/20
2040/2040 [==============================] - 13s 6ms/step - loss: 0.2264 - accuracy: 0.9392 - f1_score: 0.9390 - precision: 0.9399 - recall: 0.9383
Epoch 5/20
2040/2040 [==============================] - 12s 6ms/step - loss: 0.1892 - accuracy: 0.9504 - f1_score: 0.9504 - precision: 0.9507 - recall: 0.9500
Epoch 6/20
2040/2040 [==============================] - 13s 6ms/step - loss: 0.1693 - accuracy: 0.9542 - f1_score: 0.9542 - precision: 0.9547 - recall: 0.9538
Epoch 7/20
2040/2040 [==============================] - 1

In [35]:
for b_size, prediction in zip([32, 64, 128, 256, 512],predictions):
    print(b_size)
    print(
        pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(prediction, axis=1)), index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys())).to_markdown(tablefmt="grid")
    )
    print()
    
    # Batch Size Does not matter on the results for DNN

32
+--------------------+-----------------+----------------------+------------------+------------+
|                    |   PRED_Password |   PRED_SQL_injection |   PRED_Uploading |   PRED_XSS |
+====================+=================+======================+==================+============+
| TRUE_Password      |           21366 |                    0 |                0 |          0 |
+--------------------+-----------------+----------------------+------------------+------------+
| TRUE_SQL_injection |               7 |                 1284 |                0 |          0 |
+--------------------+-----------------+----------------------+------------------+------------+
| TRUE_Uploading     |             494 |                    0 |               74 |          0 |
+--------------------+-----------------+----------------------+------------------+------------+
| TRUE_XSS           |             152 |                    1 |              194 |       4399 |
+--------------------+---------------

In [36]:
DNN_layers = [
        keras.layers.Dense(4196, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(len(labels.keys()), activation="softmax"),
    ]
DNN_compile = {
    "loss": tf.keras.losses.CategoricalCrossentropy(), 
    "optimizer": "adam", 
    "metrics": ["accuracy", f1_score, precision, recall],
}
DNN_model = keras.Sequential(layers=DNN_layers.copy())
DNN_model.compile(**DNN_compile)
DNN_model.fit(train_X[DNN_COLS], train_Y, epochs=30, batch_size=512)

Epoch 1/30
128/128 [==============================] - 2s 7ms/step - loss: 97.7422 - accuracy: 0.7174 - f1_score: 0.7163 - precision: 0.7185 - recall: 0.7143
Epoch 2/30
128/128 [==============================] - 1s 6ms/step - loss: 0.6153 - accuracy: 0.8403 - f1_score: 0.8377 - precision: 0.8420 - recall: 0.8345
Epoch 3/30
128/128 [==============================] - 1s 6ms/step - loss: 0.5882 - accuracy: 0.8430 - f1_score: 0.8407 - precision: 0.8441 - recall: 0.8386
Epoch 4/30
128/128 [==============================] - 1s 6ms/step - loss: 0.5533 - accuracy: 0.8476 - f1_score: 0.8459 - precision: 0.8498 - recall: 0.8424
Epoch 5/30
128/128 [==============================] - 1s 6ms/step - loss: 0.4986 - accuracy: 0.8591 - f1_score: 0.8583 - precision: 0.8619 - recall: 0.8548
Epoch 6/30
128/128 [==============================] - 1s 6ms/step - loss: 2.3433 - accuracy: 0.7766 - f1_score: 0.7754 - precision: 0.7784 - recall: 0.7726
Epoch 7/30
128/128 [==============================] - 1s 6ms/st

In [37]:
DNN_prediction = DNN_model.predict(test_X[DNN_COLS])
print(
        pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(DNN_prediction, axis=1)), index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys())).to_markdown(tablefmt="grid")
    )

875/875 [==============================] - 1s 1ms/step
+--------------------+-----------------+----------------------+------------------+------------+
|                    |   PRED_Password |   PRED_SQL_injection |   PRED_Uploading |   PRED_XSS |
+====================+=================+======================+==================+============+
| TRUE_Password      |           21366 |                    0 |                0 |          0 |
+--------------------+-----------------+----------------------+------------------+------------+
| TRUE_SQL_injection |             767 |                  524 |                0 |          0 |
+--------------------+-----------------+----------------------+------------------+------------+
| TRUE_Uploading     |             568 |                    0 |                0 |          0 |
+--------------------+-----------------+----------------------+------------------+------------+
| TRUE_XSS           |             544 |                    1 |                0 

In [38]:
precision_score(np.argmax(test_Y, axis=1), np.argmax(DNN_prediction, axis=1), average="macro")

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.7293151625029448

In [39]:
recall_score(np.argmax(test_Y, axis=1), np.argmax(payload_prediction, axis=1), average="macro")

0.7498946481247366

In [40]:
pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(DNN_prediction, axis=1)), index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys()))

,PRED_Password,PRED_SQL_injection,PRED_Uploading,PRED_XSS
TRUE_Password,21366,0,0,0
TRUE_SQL_injection,767,524,0,0
TRUE_Uploading,568,0,0,0
TRUE_XSS,544,1,0,4201


In [41]:
pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(file_data_prediction, axis=1)),index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys()))

,PRED_Password,PRED_SQL_injection,PRED_Uploading,PRED_XSS
TRUE_Password,21366,0,0,0
TRUE_SQL_injection,0,1291,0,0
TRUE_Uploading,0,0,568,0
TRUE_XSS,4419,0,0,327


In [111]:
result = np.array([([d1,d2,d3,d4] if max([d1,d2,d3,d4])==d4 else [p1, p2 ,p3, p4] if max([p1, p2, p3, p4]) in [p2, p3, p4] else [d1, 0, 0, d4]) for (d1,d2,d3,d4),(p1,p2,p3,p4) in zip(DNN_prediction, file_data_prediction)])
result_weight = np.array([([d1,d2,d3,d4] if max([d1,d2,d3,d4])==d4 else [p1, p2 ,p3, p4] if max([p1, p2, p3, p4]) in [p2, p3, p4] else [d1, 0, 0, d4*2]) for (d1,d2,d3,d4),(p1,p2,p3,p4) in zip(DNN_prediction, file_data_prediction)])

In [117]:
print("result - default")
precision_Score, recall_Score, f1_Score = precision_score(np.argmax(test_Y, axis=1), np.argmax(result, axis=1), average="macro"),\
    recall_score(np.argmax(test_Y, axis=1), np.argmax(result, axis=1), average="macro"), \
        f1_score(np.argmax(test_Y, axis=1), np.argmax(result, axis=1), average="macro")
        
print(f"""{precision_Score=}\t{recall_Score=}\t{f1_Score=}""")
print(
pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(result, axis=1)), index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys())).to_markdown(tablefmt="grid")
)
print()
print()
print("result-weighted")
precision_Score, recall_Score, f1_Score = precision_score(np.argmax(test_Y, axis=1), np.argmax(result_weight, axis=1), average="macro"),\
    recall_score(np.argmax(test_Y, axis=1), np.argmax(result_weight, axis=1), average="macro"), \
        f1_score(np.argmax(test_Y, axis=1), np.argmax(result_weight, axis=1), average="macro")
        
print(f"""{precision_Score=}\t{recall_Score=}\t{f1_Score=}""")
print(
pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(result_weight, axis=1)), index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys())).to_markdown(tablefmt="grid")
)

result - default
precision_Score=0.9937816621788143	recall_Score=0.971291613990729	f1_Score=0.9816231081749397
+--------------------+-----------------+----------------------+------------------+------------+
|                    |   PRED_Password |   PRED_SQL_injection |   PRED_Uploading |   PRED_XSS |
+====================+=================+======================+==================+============+
| TRUE_Password      |           21366 |                    0 |                0 |          0 |
+--------------------+-----------------+----------------------+------------------+------------+
| TRUE_SQL_injection |               0 |                 1291 |                0 |          0 |
+--------------------+-----------------+----------------------+------------------+------------+
| TRUE_Uploading     |               0 |                    0 |              568 |          0 |
+--------------------+-----------------+----------------------+------------------+------------+
| TRUE_XSS           |   

In [104]:
PassWord_XSS_test_Y = []
PassWord_XSS_fit_Y = []
for pred, test in zip(result, test_Y):
    if not (test[1] + test[2]):
        PassWord_XSS_test_Y.append(test)
        PassWord_XSS_fit_Y.append(pred)


In [105]:
precision_score(np.argmax(PassWord_XSS_test_Y, axis=1), np.argmax(PassWord_XSS_fit_Y, axis=1), average="macro"),\
    recall_score(np.argmax(PassWord_XSS_test_Y, axis=1), np.argmax(PassWord_XSS_fit_Y, axis=1), average="macro"), \
        f1_score(np.argmax(PassWord_XSS_test_Y, axis=1), np.argmax(PassWord_XSS_fit_Y, axis=1), average="macro")

(0.990989587375283, 0.9592054226068734, 0.9741960846502298)